# 🔍 Análise da Qualidade dos Dados - Camada Silver

Este notebook tem como objetivo avaliar a qualidade dos dados presentes na tabela `silver_criptoativos`. Serão verificadas:
- Presença de valores nulos
- Valores fora do domínio esperado
- Duplicidade de registros
- Distribuição de dados para atributos principais

In [0]:
from pyspark.sql.functions import col, count, isnan, when, approx_count_distinct, min, max

In [0]:
# 📥 Leitura da tabela Silver
silver_df = spark.read.table("lakehouse.silver.cripto")
display(silver_df.limit(5))

In [0]:
# 📊 1. Verificação de valores nulos ou ausentes
nulls_df = silver_df.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in silver_df.columns
])
display(nulls_df)

In [0]:
# 🔁 2. Verificação de duplicidade de registros
# Consideramos duplicatas se todos os campos forem iguais

duplicates = silver_df.groupBy(silver_df.columns).count().filter("count > 1")
display(duplicates)

In [0]:
# 🧪 3. Verificação de valores fora do domínio esperado

# Preço deve ser ≥ 0
silver_df.filter(col("price_usd") < 0).display()

# Timestamp não pode estar no futuro
from pyspark.sql.functions import current_timestamp
silver_df.filter(col("timestamp") > current_timestamp()).display()

In [0]:
# 🗓️ 4. Distribuição de registros por data
from pyspark.sql.functions import to_date

silver_df.groupBy(to_date("timestamp").alias("data")).count().orderBy("data").display()

In [0]:
# 📈 5. Estatísticas básicas por atributo numérico

silver_df.select(
    count("*").alias("total_registros"),
    approx_count_distinct("asset_name").alias("ativos_distintos"),
    min("price_usd").alias("preco_min"),
    max("price_usd").alias("preco_max")
).display()